In [1]:
#Для решения задачи автоматического распознавания русской речи:
#А) Статья: https://habr.com/ru/company/ods/blog/692246/;
#Б) Открытый репо: https://github.com/openai/whisper;
#В) туториал: https://github.com/openai/whisper/blob/main/notebooks/Multilingual_ASR.ipynb
#Г) Решение универсально, работает для любого языка, см. документацию в А).

import warnings
warnings.simplefilter('ignore')
                        
import torch
import whisper
from whisper.utils import format_timestamp

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium") # 1 раз скачать - дальше можно пользоваться без интернета
options = dict(language='ru', beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)

## ex1

In [30]:
dict_pred = model.transcribe('./data_ru/ex2.mp3', **transcribe_options)
#print(dict_pred.keys()) # dict_keys(['text', 'segments', 'language'])
dict_pred['text']

' Внутренний аудит – это форма контроля деятельности организации изнутри. Процедура помогает руководству проверять финансовое состояние бизнеса и достоверность отчётности.'

In [34]:
dict_pred = model.transcribe('./data_ru/ex2.mp3', **transcribe_options)

for segment in dict_pred['segments']:
    print(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {segment['text']}")

[00:00.000 --> 00:07.000]  Внутренний аудит – это форма контроля деятельности организации изнутри.
[00:07.000 --> 00:35.000]  Процедура помогает руководству проверять финансовое состояние бизнеса и достоверность отчётности.


## ex2

In [4]:
%%time
model.transcribe('./data_ru/Tovji10XSao.mp3', **transcribe_options)["text"]
# https://www.youtube.com/watch?v=Tovji10XSao

CPU times: total: 2min 3s
Wall time: 1min 51s


' Про плюсы одной из самых дорогих услуг в стоматологии вроде бы все понятно. Давайте разбираться с отрицательной стороной этой процедуры. Приветствуем! С вами экспертный журнал о стоматологии StartSmile.ru. И сейчас будем ругать имплантацию как только возможно. Не все ведь о голливудских улыбках рассказывать. Начнем с того, что операцию по имплантации зубов невозможно провести людям с абсолютными или временными противопоказаниями. И если некоторые временные ограничения, такие как беременность, удастся просто переждать, то на устранение других стоматологических проблем потребуется не только время, но и деньги. Карие с соседних зубов, плохая гигиена ротовой полости, воспаление десен, проблемы с прикусом и другие причины – все это является относительным противопоказанием к имплантации. Пока не избавитесь от подобного списка, составленного на первой консультации у имплантолога, о новых зубах придется забыть. Помимо временных, есть список абсолютных противопоказаний, при которых имплантаци

In [49]:
dict_pred = model.transcribe('./data_ru/Tovji10XSao.mp3', **transcribe_options)

for segment in dict_pred['segments']:
    start_time = format_timestamp(segment['start'], always_include_hours=True).split('.')[0]
    end_time = format_timestamp(segment['end'], always_include_hours=True).split('.')[0]
    text = f"[{start_time}-{end_time}] {segment['text']}"
    print(text)

[00:00:00-00:00:05]  Про плюсы одной из самых дорогих услуг в стоматологии вроде бы все понятно.
[00:00:05-00:00:09]  Давайте разбираться с отрицательной стороной этой процедуры.
[00:00:15-00:00:20]  Приветствуем! С вами экспертный журнал о стоматологии StartSmile.ru.
[00:00:20-00:00:24]  И сейчас будем ругать имплантацию как только возможно.
[00:00:24-00:00:31]  Не все ведь о голливудских улыбках рассказывать.
[00:00:31-00:00:39]  Начнем с того, что операцию по имплантации зубов невозможно провести людям с абсолютными или временными противопоказаниями.
[00:00:39-00:00:45]  И если некоторые временные ограничения, такие как беременность, удастся просто переждать,
[00:00:45-00:00:51]  то на устранение других стоматологических проблем потребуется не только время, но и деньги.
[00:00:51-00:01:02]  Карие с соседних зубов, плохая гигиена ротовой полости, воспаление десен, проблемы с прикусом и другие причины – все это является относительным противопоказанием к имплантации.
[00:01:02-00:01:10

## prod

In [1]:
import warnings
warnings.simplefilter('ignore')

import math
import time
import os
import shutil
import yaml
from pydub import AudioSegment

import torch
import whisper
from whisper.utils import format_timestamp
torch.cuda.empty_cache()

def create_folder(path):
    try:
        os.mkdir(path)
    except OSError as error: 
        pass
    
def delete_folder(path):
    try:
        shutil.rmtree(path)
    except FileNotFoundError as error: 
        pass

def stt_processing_1_audio(filepath, model, params, transcribe_options):
    name = filepath.split('/')[-1]
    file_format = name.split('.')[-1]
    resultpath1 = os.path.join(params['RESULTPATH'], '{}_net.txt'.format(name))
    resultpath2 = os.path.join(params['RESULTPATH'], '{}_net_ts.txt'.format(name))
    sound = AudioSegment.from_file(filepath)
    
    len_ms = len(sound)
    dt = 1000*params['PART_LEN']
    n_parts = math.ceil(len_ms/dt)
    temp_folder = './temp_folder/'
    create_folder(temp_folder)
    
    for i in range(n_parts):
        print('{}/{}'.format(i+1,n_parts))
        sound_part = sound[dt*i:dt*(i+1)]
        temp_filepath = os.path.join(temp_folder, 'part_{}.{}'.format(i,file_format))
        sound_part.export(temp_filepath, format=file_format)
        dict_pred = model.transcribe(temp_filepath, **transcribe_options)
        
        text = dict_pred["text"]
        if type(text)==list:
            text = text[0]
        with open(resultpath1, "a") as f1:
            f1.writelines(text+"\n======= {}:{}\n".format((i+1)*params['PART_LEN']//60,
                                                         (i+1)*params['PART_LEN']%60))  
        with open(resultpath2, "a") as f2:
            for segment in dict_pred['segments']:
                start_time = format_timestamp(segment['start']+i*params['PART_LEN'],
                                              always_include_hours=True).split('.')[0]
                end_time = format_timestamp(segment['end']+i*params['PART_LEN'],
                                            always_include_hours=True).split('.')[0]
                text = f"[{start_time}-{end_time}] {segment['text']}"
                f2.writelines(text+'\n')
    delete_folder(temp_folder)
    

if __name__ == "__main__":   
    with open("params.yml", "r") as stream:
        params = yaml.safe_load(stream)

    asr_model = whisper.load_model(name=params['MODEL_NAME'], device=params['DEVICE']) 
    options = dict(language=params['LANG'], beam_size=5, best_of=5)
    transcribe_options = dict(task="transcribe", **options)
    
    print('Обрабатываю файл', params['FILEPATH'])
    start_time = time.time()
    stt_processing_1_audio(params['FILEPATH'], asr_model, params, transcribe_options)
    dt = (time.time()-start_time)/60 # min
    print('Обработка завершена, обработка длилась {} минут'.format(round(dt,2)))

Обрабатываю файл ./data_ru/ex2.mp3
1/1
Обработка завершена, обработка длилась 0.13 минут
